In [2]:
import os 
os.getcwd()

'/Users/aleedom/cltk'

In [4]:
from cltk.corpus.utils.formatter import assemble_phi5_author_filepaths
from cltk.corpus.utils.formatter import phi5_plaintext_cleanup
from cltk.stem.latin.j_v import JVReplacer
from collections import Counter
from nltk.tokenize.punkt import PunktLanguageVars
import os
from cltk.lemmatize.latin.backoff import BackoffLatinLemmatizer

path = 'lexica/Lewis_Short_XML/lat.ls.perseus-eng1.xml'
path2 = 'lexica/Lewis_Short_XML/lat.ls.perseus-eng2.xml'

## Now that we've messed around a bit, why not create a filter list?

In [4]:
from collections import Counter
from cltk.corpus.utils.importer import CorpusImporter
corpus_importer = CorpusImporter('latin')
corpus_importer.list_corpora

['latin_text_perseus',
 'latin_treebank_perseus',
 'latin_text_latin_library',
 'phi5',
 'phi7',
 'latin_proper_names_cltk',
 'latin_models_cltk',
 'latin_pos_lemmata_cltk',
 'latin_treebank_index_thomisticus',
 'latin_lexica_perseus',
 'latin_training_set_sentence_cltk',
 'latin_word2vec_cltk',
 'latin_text_antique_digiliblt',
 'latin_text_corpus_grammaticorum_latinorum',
 'latin_text_poeti_ditalia',
 'latin_text_tesserae']

In [147]:
from cltk.corpus.readers import get_corpus_reader
corpus_importer.import_corpus('latin_text_perseus')
reader = get_corpus_reader(language='latin', corpus_name='latin_text_latin_library')


In [7]:
word_list = list(reader.words())

### 

### Ok so we have the component parts, we want: 
 - Total # tokens 
 - Total unique lemmata 
 - Each token as pct of tokens 
 - lemma as pct of lemmas 

In [148]:
# all words from latin library
# dict with token, count
word_count = Counter(list(reader.words()))

In [149]:
# all tokens 
word_list = list(reader.words())
import random
test = random.choices(word_list, k = 25)

In [12]:
lemmatizer = BackoffLatinLemmatizer()

In [150]:
# Get lemmata 

lemma_feeder = list(set(word_list))
unique_lemmas = []
count = 0
for word in tqdm(lemma_feeder): 
    unique_lemmas += lemmatizer.lemmatize([word.lower().strip().strip('"').strip('-')])


100%|██████████| 540429/540429 [00:12<00:00, 42378.36it/s]


In [151]:
lemma_count = Counter(unique_lemmas)

lemma_df = pd.DataFrame(lemma_count.most_common(), columns = ['lemma', 'lemma_count'])

lemma_df['pct_of_total'] = lemma_df['lemma_count'] / lemma_df['lemma_count'].sum()

In [152]:
lemma_df

,lemma,lemma_count,pct_of_total
0,"(, )",7,0.000013
1,"(ne, ne)",5,0.000009
2,"(au, au)",5,0.000009
3,"(in, in)",5,0.000009
4,"(tantum, tantus)",4,0.000007
...,...,...,...
482434,"(ecclesiastis, ecclesiastis)",1,0.000002
482435,"(absumebat, absumo)",1,0.000002
482436,"(grauame, grauame)",1,0.000002
482437,"(subsannantibus, subsanno)",1,0.000002


In [153]:
# token df 

token_df = pd.DataFrame(word_count.most_common(), columns = ['token', 'token_count'])

token_df['lemma'] = [lemmatizer.lemmatize([token.lower()])[0][1] for token in token_df['token']]

token_df = token_df[token_df['lemma'] != 'punc']

token_df['token_pct_of_total'] = token_df['token_count']/token_df['token_count'].sum() # want pct of total 

token_df = token_df.merge(lemma_df[['lemma', 'pct_of_total']], on = 'lemma', how = 'left').rename(columns = {'pct_of_total' : 'lemma_pct_of_total'})

In [154]:
count_df = token_df.groupby('lemma').token_count.sum().reset_index().sort_values('token_count', ascending = False)

count_df.head(25)

,lemma,token_count
139530,et,443730
277732,sum,433935
248486,qui,360771
170819,in,272568
181783,is,206494
210553,non,163307
163041,hic,137278
6244,-que,135343
46660,ad,132702
296065,ut,117546


In [155]:
count_df['pct_of_tokens'] = count_df['token_count'] / count_df['token_count'].sum()
count_df['cumsum'] = count_df.pct_of_tokens.cumsum()

In [156]:
count_df[(count_df['token_count'] <= 500) & (count_df['token_count'] >= 20) ]

,lemma,token_count,pct_of_tokens,cumsum
176683,inquisitio,500,0.000036,0.835095
101069,con-loco,500,0.000036,0.835131
85483,chorus,500,0.000036,0.835167
54595,alexandro,500,0.000036,0.835203
36897,69,500,0.000036,0.835239
...,...,...,...,...
101953,concup,20,0.000001,0.955594
295360,uocatio,20,0.000001,0.955596
241960,profligaris,20,0.000001,0.955597
224722,penda,20,0.000001,0.955599


In [152]:
exclude = count_df[(count_df['token_count'] <= 500) & (count_df['token_count'] >= 20) ].lemma

In [119]:
exclude

['sum',
 'et',
 'qui',
 'in',
 '-que',
 'is',
 'non',
 'hic',
 'ut',
 'cum2',
 'ego',
 'tu',
 'ad',
 '-ne',
 'ille',
 'ab',
 'si',
 'neque',
 'sed',
 'atque',
 'omnis',
 'quam',
 'possum',
 'de',
 'ipse',
 'dico',
 'quis',
 'ex',
 'res',
 'sui',
 'facio',
 'suus',
 'aut',
 'video',
 'magnus',
 'multus',
 'etiam',
 'enim',
 'habeo',
 'alius',
 'per',
 'do',
 'iam',
 'tuus',
 'deus',
 'ne',
 'meus',
 'quo',
 'idem',
 'homo',
 'autem',
 'verus',
 'tamen',
 'nos',
 'bonus',
 'quidem',
 'ita',
 'primus',
 'tantus',
 'nihil',
 nan,
 'unus',
 'venio',
 'pars',
 'nam',
 'quoque',
 'noster',
 'vel',
 'nullus',
 'nunc',
 'iste',
 'causa',
 'fero',
 'ago',
 'animus',
 'quidam',
 'sic',
 'dies',
 'modo',
 'locus',
 'publicus',
 'pater',
 'tempus',
 'corpus',
 'tum',
 'nisi',
 'fio',
 'nomen',
 'inter',
 'genus',
 'totus',
 'scribo',
 'malus',
 'e',
 'puto',
 'manus',
 'terra',
 'paruus',
 'sine',
 'pro',
 'volo',
 'inquam',
 'tam',
 'ante',
 'ubi',
 'ratio',
 'alter',
 'credo',
 'scio',
 'quia',
 

In [140]:
count_df.to_csv('latin_word_counts.csv', index = False)

## We now have a reasonable list of words to exclude 

In [12]:
from tqdm import tqdm 
import lxml.etree as ET
import pandas as pd

In [13]:
exclude = pd.read_csv('latin_word_counts.csv')

exclude = list(exclude['lemma'])

## get apuleius 

reader._fileids = ['apuleius__metamorphoses__latin.json']


## Now add definitions

In [14]:
from collections import Counter
from cltk.corpus.utils.importer import CorpusImporter
lemmatizer = BackoffLatinLemmatizer()
corpus_importer = CorpusImporter('latin')
corpus_importer.list_corpora

['latin_text_perseus',
 'latin_treebank_perseus',
 'latin_text_latin_library',
 'phi5',
 'phi7',
 'latin_proper_names_cltk',
 'latin_models_cltk',
 'latin_pos_lemmata_cltk',
 'latin_treebank_index_thomisticus',
 'latin_lexica_perseus',
 'latin_training_set_sentence_cltk',
 'latin_word2vec_cltk',
 'latin_text_antique_digiliblt',
 'latin_text_corpus_grammaticorum_latinorum',
 'latin_text_poeti_ditalia',
 'latin_text_tesserae']

In [77]:
lemmatizer.lemmatize(['ii'])[0][1]

'is'

In [178]:
from cltk.corpus.readers import get_corpus_reader
corpus_importer.import_corpus('latin_text_perseus')
reader = get_corpus_reader(language='latin', corpus_name='latin_text_perseus')


In [17]:
reader._fileids = ['apuleius__metamorphoses__latin.json']

In [94]:
lemmatizer.lemmatize(['dissimulare'])

[('dissimulare', 'dissimulo')]

In [113]:
words = list(set(reader.words()))
words = [w.lower() for w in words if w != '']
dictionary = {}
# lemmatize

final = []
count = 0
for w in tqdm(words): 
    root = lemmatizer.lemmatize([w])
    if len(root) > 1:
        print('long entry')
    dictionary[root[0][1]] = {} 
    dictionary[root[0][1]]['token']  = root[0][0]
    dictionary[root[0][1]]['lemma'] = root[0][1]

    

100%|██████████| 18648/18648 [00:00<00:00, 58937.33it/s]


In [117]:
len(dictionary.keys())

8167

In [111]:
delete = [key for key in dictionary.keys() if key in exclude]
for key in tqdm(delete): 
    del dictionary[key]

100%|██████████| 8166/8166 [00:00<00:00, 1190582.82it/s]


In [112]:
dictionary

{'punc': {'token': ')', 'lemma': 'punc'}}

In [19]:
final = []
for word in out: 
    lemma = lemmatizer.lemmatize([word])
    final.append(lemma[0][1])

In [302]:
dictionary

{'doceo': {'token': 'docebat', 'lemma': 'doceo'},
 'incerto2': {'token': 'incertans', 'lemma': 'incerto2'},
 'internuntium': {'token': 'internuntia', 'lemma': 'internuntium'},
 'produco': {'token': 'producunt', 'lemma': 'produco'},
 'crinis': {'token': 'crines', 'lemma': 'crinis'},
 'immemor': {'token': 'immemor', 'lemma': 'immemor'},
 'reddo': {'token': 'reddituram', 'lemma': 'reddo'},
 'nomen': {'token': 'nomen', 'lemma': 'nomen'},
 'aurigo': {'token': 'auriga', 'lemma': 'aurigo'},
 'teneo': {'token': 'tenebat', 'lemma': 'teneo'},
 'disturbatae': {'token': 'disturbatae', 'lemma': 'disturbatae'},
 'factio': {'token': 'factionibus', 'lemma': 'factio'},
 'commoveo': {'token': 'commoti', 'lemma': 'commoveo'},
 'cibatus': {'token': 'cibatui', 'lemma': 'cibatus'},
 'aegroto': {'token': 'aegroto', 'lemma': 'aegroto'},
 'cerberum': {'token': 'cerberum', 'lemma': 'cerberum'},
 'insuper': {'token': 'insuper', 'lemma': 'insuper'},
 'emcrendis': {'token': 'emcrendis', 'lemma': 'emcrendis'},
 'no

In [304]:
test = random.sample(list(dictionary.keys()), k = 10)

In [317]:
count = 0
tree = ET.parse(path)
entries = tree.xpath('//entryFree')
d = {}
for word in tqdm(dictionary.keys()): 
    senses = []
    endings = []
    quotes = []
    gender = ''
    for entry in entries: 
        d[word] = {}
        if entry.get('type') != 'main': 
            pass
        if entry.get('key') == word: 
            if entry.find('itype') is not None:
                endings.append(entry.find('itype').text)
            if entry.find('gen') is not None: 
                gender = entry.find('gen').text
            for sense in entry.findall('sense'):
                count = 0
#                 print(sense.get('n'), sense.get('level'))
                for hi in sense.findall('hi'):
                    if hi.text is not None:
                        senses.append(f'Sense {sense.get("n")}: {hi.text}')
                        count += 1
                    if count == 5: 
                        continue
                for cit in sense.findall('cit'):
                    count = 0
                    for quote in cit.findall('quote'):
                        if quote.text is not None:
                            quotes.append(f'Sense {sense.get("n")}: {quote.text}') 
        d[word]['senses']= "<br>".join(senses)
        d[word]['endings'] = ' -'.join(endings)
        d[word]['gender'] = gender
        d[word]['sentences'] = '<br>'.join(quotes)

100%|██████████| 8167/8167 [14:11<00:00,  9.60it/s]


In [316]:
d

{'juro': {'senses': 'Sense I: a.<br>Sense I: to swear, to take an oath<br>Sense I: absol.<br>Sense I: inf.<br>Sense I: inf.<br>Sense I: inf.<br>Sense I: pass.<br>Sense I: to vow<br>Sense I: promise to one<br>Sense I: to swear with certain words<br>Sense I: to take a prescribed form of oath<br>Sense I: de<br>Sense I: Pass. impers.<br>Sense II: to conspire<br>Sense II: inf.<br>Sense A: Pass.<br>Sense 1: Called upon<br>Sense 1: taken to witness in an oath<br>Sense 2: Under an oath, bound by an oath<br>Sense B: Act., having sworn, that has sworn<br>Sense B: Sup.<br>Sense B: the most trustworthy<br>Sense B: Adv.<br>Sense B: with an oath, under oath',
  'endings': 'āvi, ātum, 1',
  'gender': '',
  'sentences': 'Sense I: cui si aram tenens juraret, crederet nemo,<br>Sense I: cum ille mihi nihil, nisi ut jurarem, permitteret,<br>Sense I: cum enim faciles sint nonnulli hominum ad jurandum,<br>Sense I: posteaquam juratum est, denegatur actio,<br>Sense I: ex animi tui sententia jurāris,<br>Sense 

In [318]:
eligible = count_df[(count_df['cumsum'] > .8) & (count_df['token_count'] >=2)]

elig = list(eligible.lemma)

delete = [k for k in d.keys() if k not in elig]
    



## Try and make a catalog

In [211]:
catalog = pd.DataFrame(reader.fileids(), index = None, columns = ['source'])

catalog['title'] = catalog.source.apply(lambda x: x.split('__')[1])

catalog

In [263]:
latin_library = get_corpus_reader('latin_text_latin_library', 'latin')

ll = pd.DataFrame(latin_library.fileids(), index = None, columns = ['source'])

ll['author'] = ll.source.apply(lambda x: x.split('/')[0])

ll['title'] = ll.source.apply(lambda x: x.split('/')[1])

ll['title'] = ll.source.apply(lambda x: x.split('/')[1] if len(x.split('/')) >1 else x.split('/')[0].strip('.txt'))

ll.title = ll.title.str.replace('.txt', '')

cat = pd.concat([catalog, ll])

cat.author = cat.author.str.replace('.txt', '')

cat.to_csv('data/latin_catalog.csv', index = False)

## Ok now we have a catalog that includes the source column

### Now we can wrap everything up and include the data from the Greek project. 

### If we really want to we could even combine the two

In [264]:
%%capture
!pip install genanki 
import genanki

In [299]:
d['ancoral']['senses']

''

In [327]:
# make flashcards for apuleius 

def make_flashcards(source: 'dict', deck_title: 'str' = '', model_name = ''): 
        """
        Makes an Anki deck and saves the .apkg file in the workind directory. 
        :param deck_title: title of the deck. If not specified, defaults to '{author} - {title} {vocab}'
        :param deck_description: description of the deck. 
        :param deck_id: deck id. Defaults to a random 10 digit integer 
        :param css: optional css styling for the cards. Defaults to 
        
                .card {
         font-family: times;
         font-size: 30px;
         text-align: center;
         color: black;
         background-color: white;
        }

        .card1 { background-color:Lavender }
        
        :param model: specify a model for the cards you'd like to generate. Defaults to 
        
        genanki.Model(model_id, 'Simple Model', fields=[
        {'name': 'Question'},
        {'name': 'Answer'},
        {'name' : 'Sentences'}
        ],
        templates=[
        {'name': 'Card 1',
        'qfmt': '{{Question}}',
        'afmt': '{{FrontSide}}<hr id="answer">{{Answer}}<br></br> {{Sentences}}',
        }
        ],
        css = style)
        """
        
        default_style = """
        .card {
        font-family: sans-serif;
        font-size: 30px;
        text-align: center;
        color: black;
        background-color: white;}
        
        .card1 { background-color:Lavender }
        """
        
        model_id = random.randrange(1<<30 , 1 << 31)
        
        deck_id = random.randrange(1<<30 , 1 << 31)
        
        css = default_style 
        
        model = genanki.Model(model_id, f'{model_name} Flashcard Model',
                              fields=[
                                  {'name': 'Key'},
                                  {'name' : 'Endings'},
                                  {'name' : 'Gender'},
                                  {'name': 'Answer'},
                                  {'name' : 'Sentences'}
                              ],
                              templates=[
                                  {
                                      'name': 'Card 1',
                                      'qfmt': '{{Key}} -{{Endings}} {{Gender}}',
                                      'afmt': '''{{FrontSide}}<hr id="answer"> 
                                      {{Answer}}
                                      <br></br>
                                      {{Sentences}}''',
                                  }
                              ],
                              css = css)
        
        deck = genanki.Deck(deck_id = deck_id, 
                      name = deck_title, 
                      description = f'{deck_title}') 
        
        # clear empty notes 
    
        word_list = [k for k in source.keys() if source[k]['sentences'] != '']
        
       
        # hold entries for notes 
        
        base = {} 
        
        for word in tqdm(word_list):
            base[word] = {}
            ### new way
            entry = source[word]
                # no need to keep going 
            
            endings = entry['endings']
            gender = entry['gender']
#             tenses = '<br>'.join(entry['tenses'])
            # get front string

            front_string = \
            f"""{word}, {str(endings)}, {str(gender)}"""

            # get back string
            answers = entry['senses']
            try:
                sentences = entry['sentences']
            except KeyError:
                print('key error with: {}'.format(entry))
                pass


            
        
                        
        # make cards 
        # note that you have to control styling with Anki templates \
        # and that Anki won't respect Python string formatting 
        
            if gender == {}: 
                gender = ''
            card = genanki.Note(
                model = model, 
                fields = [
                    word,
                    endings, 
                    gender,
                    answers,
                    sentences
                ]
            )
        
            deck.add_note(card)
            
        
        # export deck 
        try:
            genanki.Package(deck).write_to_file(f'{deck_title}.apkg')
        except TypeError: 
                print('error on {}'.format((word, endings, gender, tenses,
                                           answers, sentences)))

In [328]:
make_flashcards(source = d, deck_title='Apuleius Metamorphoses Vocab')

100%|██████████| 5119/5119 [00:00<00:00, 98397.11it/s]
